In [1]:
# %export CUDA_VISIBLE_DEVICES='0'
import os 

os.environ['CUDA_VISIBLE_DEVICES']= '0'

In [5]:
os.getenv('CUDA_VISIBLE_DEVICES')

'0'

In [3]:
!nvidia-smi

Sat Dec  9 12:30:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  Off  | 00000000:34:00.0 Off |                    0 |
| N/A   31C    P0    50W / 350W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  Off  | 00000000:36:00.0 Off |                    0 |
| N/A   

In [6]:
%pip install datasets transformers nltk

  Using cached datasets-2.15.0-py3-none-any.whl.metadata (20 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached pyarrow-14.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached xxhash-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.15-py311-none-any.whl.metadata (7.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
Using cached datasets-2.15.0-py3-none-any.whl (521 kB)
Using cached dill-0.3.7-py3-none-any.whl (115 kB)
Using cached pyarrow-14.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.1 MB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached multiprocess-0.70.15-py311-

In [7]:
!pip install transformers[torch]

  Using cached accelerate-0.25.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


In [8]:
!pip install accelerate -U

In [3]:
import transformers
from datasets import load_dataset, load_metric

In [6]:
data = load_dataset("csv",data_files="dataset/data.csv")
data


DatasetDict({
    train: Dataset({
        features: ['tools', 'question', 'answer'],
        num_rows: 264
    })
})

In [7]:
datasets_train_test = data["train"].train_test_split(test_size=72)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=10)

data["train"] = datasets_train_validation["train"]
data["validation"] = datasets_train_validation["test"]
data["test"] = datasets_train_test["test"]
data

DatasetDict({
    train: Dataset({
        features: ['tools', 'question', 'answer'],
        num_rows: 182
    })
    validation: Dataset({
        features: ['tools', 'question', 'answer'],
        num_rows: 10
    })
    test: Dataset({
        features: ['tools', 'question', 'answer'],
        num_rows: 72
    })
})

In [2]:
from transformers import AutoTokenizer


In [10]:
model_checkpoint = "t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/raid/test1/anaconda3/envs/devrevenv/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [11]:
prefix = "question: "
max_input_length = 512
max_target_length = 256

prompt = '\n\n'

def preprocess_data(examples):
  inputs = [ tools + prompt + prefix + question for tools, question in zip(examples["tools"],examples["question"])]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["answer"], max_length=max_target_length,truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [12]:
tokenized_datasets = data.map(preprocess_data, batched=True)
tokenized_datasets

Map:   0%|          | 0/182 [00:00<?, ? examples/s]/raid/test1/anaconda3/envs/devrevenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 72/72 [00:00<00:00, 937.84 examples/s]


DatasetDict({
    train: Dataset({
        features: ['tools', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 182
    })
    validation: Dataset({
        features: ['tools', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    test: Dataset({
        features: ['tools', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 72
    })
})

In [1]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

/raid/test1/anaconda3/envs/devrevenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
batch_size = 8
model_name = "t5-large"
model_dir = f"models/t5/{model_name}"
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')

# metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [20]:
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /raid/test1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
%conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Channels:
 - pytorch
 - nvidia
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /raid/test1/anaconda3/envs/devrevenv

  added / updated specs:
    - pytorch
    - pytorch-cuda=12.1
    - torchaudio
    - torchvision


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2023.11.1~ --> pkgs/main/linux-64::certifi-2023.11.17-py311h06a4308_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install tensorboardX

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached protobuf-4.25.1-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)
Using cached protobuf-4.25.1-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
# Function that returns an untrained model to be trained

def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=115, training_loss=0.0, metrics={'train_runtime': 71.2694, 'train_samples_per_second': 12.768, 'train_steps_per_second': 1.614, 'total_flos': 1970193039360000.0, 'train_loss': 0.0, 'epoch': 5.0})

In [26]:
trainer.save_model()

In [22]:
model_name = "t5-large"
model_dir = f"models/t5/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [23]:
import torch

# get test split
test_tokenized_dataset = tokenized_datasets["test"]

# pad texts to the same length
def preprocess_test(examples):
  inputs = [ tools + prompt + prefix + question for tools, question in zip(examples["tools"],examples["question"])]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                           padding="max_length")
  return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=18)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
# all_titles = tokenizer(test_tokenized_dataset["answer"], max_length=max_target_length,
#                        truncation=True, padding="max_length")["input_ids"]

# # compute metrics
# predictions_labels = [all_predictions_flattened, all_titles]
# print(predictions_labels)
# compute_metrics(predictions_labels)

Map: 100%|██████████| 72/72 [00:00<00:00, 664.73 examples/s]
/raid/test1/anaconda3/envs/devrevenv/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [24]:
len(all_predictions_flattened)

72

In [25]:
import nltk

In [27]:
for i in range(72):
    decoded_output = tokenizer.batch_decode(all_predictions_flattened[i], skip_special_tokens=False)
    print("i is ",i,decoded_output)
# predicted_title = nltk.sent_tokenize(decoded_output.strip())

# print(predicted_title)

i is  0 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tickets', 'that', 'need', '', 'a', 'response', '",', '"', 'type', '"', ':', '"', 'arra']
i is  1 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tickets', 'that', 'need', '', 'a', 'response', '",', '"', 'type', '"', ':', '"', 'arra']
i is  2 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tickets', 'that', 'need', '', 'a', 'response', '",', '"', 'type', '"', ':', '"', 'arra']
i is  3 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tickets', 'that', 'need', '', 'a', 'response', '",', '"', 'type', '"', ':', '"', 'arra']
i is  4 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tickets', 'that', 'need', '', 'a', 'response', '",', '"', 'type', '"', ':', '"', 'arra']
i is  5 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tickets', 'that', 'need', '', 'a', 'response', '",', '"', 'type', '"', ':', '"', 'arra']
i is  6 ['<pad>', '<extra_id_0>', '"', 'F', 'il', 'ters', 'for', 'tick